In [1]:
import requests
import json
import pandas as pd


In [3]:
headers = {
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Opera GX";v="114"',
    'Referer': 'https://www.wildberries.ru/catalog/0/search.aspx?search=%D0%B3%D0%B8%D1%80%D0%BB%D1%8F%D0%BD%D0%B4%D0%B0%20%D1%88%D1%82%D0%BE%D1%80%D0%B0',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 OPR/114.0.0.0',
    'sec-ch-ua-platform': '"Windows"',
}

response = requests.get('https://static-basket-01.wbbasket.ru/vol2/site/t/jst/catalogCard.eb449400.json', headers=headers)
with open('data.json', 'w', encoding='utf-8') as file:
    json.dump(response.json(), file)

In [ ]:
with open("data.json", 'r') as file:
    data = json.load(file)

